In [1]:
# define some input data locations

from os.path import join
path = '/home/nils/data/EUR-44'
files = ['tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19710101-19751231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19760101-19801231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19810101-19851231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19860101-19901231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19910101-19951231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19960101-20001231.nc',]
resource = [ join(path,nc) for nc in files ]

Execution with plain python (no birdhouse components included )

In [2]:
import icclim 

from ocgis import RequestDataset, OcgOperations, env
from ocgis.contrib.library_icclim import IcclimTG90p

env.OVERWRITE = True
########################################################################################################################
# Compute a custom percentile basis using ICCLIM. ######################################################################
########################################################################################################################

# Subset the input dataset to return the desired base period for the percentile basis.
variable = 'tas'
years = range(1971, 2001)
time_region = {'year': years}

rd = RequestDataset(uri=resource, variable=variable)
field = rd.get()
field.get_time_region(time_region)

# Calculate the percentile basis. The data values must be a three-dimensional array.
arr = field.variables[variable].value.squeeze()
dt_arr = field.temporal.value_datetime
percentile = 90
window_width = 5
percentile_dict = IcclimTG90p.get_percentile_dict(arr, dt_arr, percentile, window_width)

calc = [{'func': 'icclim_TG90p', 'name': 'TG90p', 'kwds': {'percentile_dict': percentile_dict}}]
calc_grouping = 'year'
ops = OcgOperations(dataset=rd, calc=calc, calc_grouping=calc_grouping, output_format='nc')
coll = ops.execute()
print coll

/home/nils/sandbox/notebooks/ocgis_output/ocgis_output.nc


In [3]:
from flyingpigeon import indices as ind

In [14]:
ind.calc_indice_percentile?

In [20]:
reload(ind)

nc_indice = ind.calc_indice_percentile(resources=resource, variable='tas', indices='TG90p',
                       period=(1980,1990), percentile=95, groupings='yr', prefix='test2', dir_output='/home/nils/tests')
print nc_indice

['/home/nils/sandbox/notebooks/TG95p_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19710101-20001231.nc']


Execute WPS icclim processes with PHYTHON 

In [21]:
from owslib.wps import WebProcessingService, monitorExecution
wps = WebProcessingService(url="http://localhost:8093/wps", verbose=False, skip_caps=False)

Explore some informations:

In [22]:
wps.identification.title

'Flyingpigeon'

In [23]:
for process in wps.processes:
    process.identifier, process.title

In [24]:
for process in wps.processes:
    print '%s \t : %s '% (process.identifier, process.abstract) 

subset_continents 	 : Returns only the selected polygon for each input dataset 
subset_countries 	 : Returns only the selected polygon for each input dataset 
subset_regionseurope 	 : Returns only the selected polygon for each input dataset 
extractpoints 	 : Extract Timeseries for specified coordinates from grid data 
indices_single 	 : This process calculates climate indices based on one single input variable. 
indices_percentile 	 : Calculation of climate indices based on one single input variable and based on percentils of a referece period. 
weatherregimes_obs 	 : Weather Regimes based on pressure patterns, fetching selected Realayses Datasets 
weatherregimes 	 : Weather Regimes based on pressure patterns (kmean method) 
robustness 	 : Calculates the robustness as the ratio of noise to signal in an ensemle of timeseries 
analogs 	 : Search for days with analog pressure pattern 
segetalflora 	 : Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EU

In [27]:
execute = wps.execute(
    identifier="indices_percentile",
    inputs=[
    ('indices','TG90p'),
    ('groupings','yr'),
    ('resource',resource[0]),
    ('resource',resource[1]),
    ('resource',resource[2]),
    ('resource',resource[3]),
    ('resource',resource[4]),
    ('resource',resource[5]),
    ], async=True
    )
# check process if completed ...

monitorExecution(execute, sleepSecs=5)
o = execute.processOutputs[0]
print o.reference

 owslib.wps.WPSException : {'locator': 'None', 'code': 'NoApplicableCode', 'text': "[Errno 2] No such file or directory: './pywpsInputFwqJnh.base64'"}


IndexError: list index out of range